In [14]:
import pandas as pd
df = pd.read_csv('../data/raw/NBAodds2025-11-10.csv')
df.head(5)

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmakers
0,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
1,3f7e2359b4a5532aafa274ff22714d51,basketball_nba,NBA,2025-11-11T00:10:00Z,Charlotte Hornets,Los Angeles Lakers,"[{'key': 'fanduel', 'title': 'FanDuel', 'last_..."
2,6fb85302d766cc4bfcd431455d308748,basketball_nba,NBA,2025-11-11T00:10:00Z,Orlando Magic,Portland Trail Blazers,"[{'key': 'fanduel', 'title': 'FanDuel', 'last_..."
3,bc264cf0d07f01c785e02a8b9e27f59b,basketball_nba,NBA,2025-11-11T00:30:00Z,Miami Heat,Cleveland Cavaliers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
4,404b446a92bde56ee3697b9a2695e468,basketball_nba,NBA,2025-11-11T01:10:00Z,Chicago Bulls,San Antonio Spurs,"[{'key': 'fanduel', 'title': 'FanDuel', 'last_..."


In [3]:
df.columns, df.shape, df['bookmakers'].isna().sum()

(Index(['id', 'sport_key', 'sport_title', 'commence_time', 'home_team',
        'away_team', 'bookmakers'],
       dtype='object'),
 (12, 7),
 np.int64(0))

In [4]:
df['bookmakers'][0]

"[{'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2025-11-10T19:28:57Z', 'markets': [{'key': 'spreads', 'last_update': '2025-11-10T19:28:57Z', 'outcomes': [{'name': 'Detroit Pistons', 'price': 1.89, 'point': -11.5}, {'name': 'Washington Wizards', 'price': 1.93, 'point': 11.5}]}]}, {'key': 'lowvig', 'title': 'LowVig.ag', 'last_update': '2025-11-10T19:30:31Z', 'markets': [{'key': 'spreads', 'last_update': '2025-11-10T19:30:31Z', 'outcomes': [{'name': 'Detroit Pistons', 'price': 1.9, 'point': -11.5}, {'name': 'Washington Wizards', 'price': 1.99, 'point': 11.5}]}]}, {'key': 'betonlineag', 'title': 'BetOnline.ag', 'last_update': '2025-11-10T19:30:20Z', 'markets': [{'key': 'spreads', 'last_update': '2025-11-10T19:30:20Z', 'outcomes': [{'name': 'Detroit Pistons', 'price': 1.87, 'point': -11.5}, {'name': 'Washington Wizards', 'price': 1.95, 'point': 11.5}]}]}, {'key': 'mybookieag', 'title': 'MyBookie.ag', 'last_update': '2025-11-10T19:29:58Z', 'markets': [{'key': 'spreads', 'last_

Bookmakers column is serialized, address below.

In [5]:
import ast #abstract syntax tree

def safe_parse(val):
    if isinstance(val, (list, dict)):
        return val
    try:
        return ast.literal_eval(val)
    except Exception:
        return None

# parse safely
df['bookmakers'] = df['bookmakers'].apply(safe_parse)

# ensure every row has either a list or an empty list
df['bookmakers'] = df['bookmakers'].apply(lambda x: x if isinstance(x, list) else [])

# now explode while preserving id alignment
exploded = df.explode('bookmakers', ignore_index=True)

# normalize the nested bookmaker dicts
bookmakersdf = pd.json_normalize(exploded['bookmakers'])

# attach id directly from exploded version
bookmakersdf['id'] = exploded['id'].values
bookmakersdf['id'].nunique(), bookmakersdf['id'].isna().sum()


(12, np.int64(0))

In [6]:
bookmakersdf.sample(15)

,key,title,last_update,markets,id
39,betanysports,BetAnything,2025-11-10T19:30:21Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",6fb85302d766cc4bfcd431455d308748
117,betparx,betPARX,2025-11-10T19:30:20Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",ed82190bf1c73a6310c46ee399696436
134,ballybet,Bally Bet,2025-11-10T19:30:07Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",f557c9173a46a3c7c87afb4bd68f6644
13,bovada,Bovada,2025-11-10T19:30:34Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",f2e9b932ad12be3f2f47166b3eb0d456
72,ballybet,Bally Bet,2025-11-10T19:30:07Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",404b446a92bde56ee3697b9a2695e468
69,betanysports,BetAnything,2025-11-10T19:30:21Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",404b446a92bde56ee3697b9a2695e468
99,betparx,betPARX,2025-11-10T19:30:20Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",eea6f48cf04bc26e631209e50a4a3af1
129,ballybet,Bally Bet,2025-11-10T19:30:07Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",1712f1f48109a3c9ef509a77914aaa06
84,hardrockbet,Hard Rock Bet,2025-11-10T19:29:57Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",4f07ee458d197bc2ccdcfcd9934ffc03
96,betus,BetUS,2025-11-10T19:30:08Z,"[{'key': 'spreads', 'last_update': '2025-11-10...",eea6f48cf04bc26e631209e50a4a3af1


In [7]:
bookmakersdf['markets'][0]

[{'key': 'spreads',
  'last_update': '2025-11-10T19:28:57Z',
  'outcomes': [{'name': 'Detroit Pistons', 'price': 1.89, 'point': -11.5},
   {'name': 'Washington Wizards', 'price': 1.93, 'point': 11.5}]}]

Now lets further unpack the markets column

In [8]:
bookmakersdf['markets'] = bookmakersdf['markets'].apply(safe_parse)
markets_explosion = bookmakersdf.explode('markets', ignore_index=True)

markets_explosion.head()


,key,title,last_update,markets,id
0,draftkings,DraftKings,2025-11-10T19:28:57Z,"{'key': 'spreads', 'last_update': '2025-11-10T...",f2e9b932ad12be3f2f47166b3eb0d456
1,lowvig,LowVig.ag,2025-11-10T19:30:31Z,"{'key': 'spreads', 'last_update': '2025-11-10T...",f2e9b932ad12be3f2f47166b3eb0d456
2,betonlineag,BetOnline.ag,2025-11-10T19:30:20Z,"{'key': 'spreads', 'last_update': '2025-11-10T...",f2e9b932ad12be3f2f47166b3eb0d456
3,mybookieag,MyBookie.ag,2025-11-10T19:29:58Z,"{'key': 'spreads', 'last_update': '2025-11-10T...",f2e9b932ad12be3f2f47166b3eb0d456
4,fliff,Fliff,2025-11-10T19:29:56Z,"{'key': 'spreads', 'last_update': '2025-11-10T...",f2e9b932ad12be3f2f47166b3eb0d456


In [13]:
marketsdf = pd.json_normalize(markets_explosion['markets'])
#marketsdf['bookmaker_id'] = markets_explosion['id']
marketsdf.head()

,key,last_update,outcomes
0,spreads,2025-11-10T19:28:57Z,"[{'name': 'Detroit Pistons', 'price': 1.89, 'p..."
1,spreads,2025-11-10T19:30:31Z,"[{'name': 'Detroit Pistons', 'price': 1.9, 'po..."
2,spreads,2025-11-10T19:30:20Z,"[{'name': 'Detroit Pistons', 'price': 1.87, 'p..."
3,spreads,2025-11-10T19:29:58Z,"[{'name': 'Detroit Pistons', 'price': 1.91, 'p..."
4,spreads,2025-11-10T19:29:56Z,"[{'name': 'Detroit Pistons', 'price': 1.91, 'p..."


In [10]:
markets_explosion.shape, marketsdf.shape, bookmakersdf.shape

((143, 5), (143, 4), (143, 5))